In [1]:
from google.cloud import bigquery
project_id = "prod-ai-project"
client = bigquery.Client(project=project_id)

sql = """select series_id, episode_id, org_input_text, org_output_text, prompt 
        from webtoon_translation.structured_240820_ep_line
        where data_split = 'romance_valid'"""
df = client.query(sql).result().to_dataframe()

answers = df['org_output_text']

/home/bun.2/.cache/pypoetry/virtualenvs/poetry-env-eDEwtiIl-py3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [2]:
print(len(answers))

74


In [3]:
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate import bleu
import argparse
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

def compute_bleu(labels, preds):
    scores = []
    smoothie = SmoothingFunction().method4
    for label, pred in zip(labels, preds):
        scores.append(bleu([label.strip().upper()], pred.strip().upper(), smoothing_function=smoothie))
    return np.mean(scores)
    

In [4]:
f_bleu = []
r_bleu = []
for data_idx in range(10,11):#len(answers)):
    pred_df = pd.read_csv(f"data/simple_testset/result_{data_idx}.csv")
    first = pred_df['first_translate']  
    review = pred_df['second_translate']
    first_bleu = compute_bleu(answers[data_idx], first)
    review_bleu = compute_bleu(answers[data_idx], review)
    f_bleu.append(first_bleu)
    r_bleu.append(review_bleu)
    #print(first_bleu, review_bleu)
print(round(np.mean(f_bleu),3), round(np.mean(r_bleu),3))

0.435 0.412


In [45]:
#관용어 추가이후 블루 스코어 확인 코드
f_bleu = []
r_bleu = []
for data_idx in range(58,68):#len(answers)):
    pred_df = pd.read_csv(f"data/simple_testset/idiom_result_{data_idx}.csv")
    first = pred_df['first_translate']  
    review = pred_df['idiom_translate']
    first_bleu = compute_bleu(answers[data_idx], first)
    review_bleu = compute_bleu(answers[data_idx], review)
    f_bleu.append(first_bleu)
    r_bleu.append(review_bleu)
    #print(first_bleu, review_bleu)
print(round(np.mean(f_bleu),3), round(np.mean(r_bleu),3))

0.525 0.521


In [5]:
#검수 작업 이후 블루 스코어 확인 코드
for data_idx in range(10,11):#len(answers)):
    pred_df = pd.read_csv(f"data/simple_testset/result_{data_idx}.csv")
    hangles = pred_df['hangle']
    preds_first = pred_df['first_translate']  
    preds_second = pred_df['second_translate']
    labels = answers[data_idx]
    smoothie = SmoothingFunction().method4
    for label, pred_f, pred_s, hangle in zip(labels, preds_first, preds_second, hangles):
        first_bleu = bleu([label.strip().upper()], pred_f.strip().upper(), smoothing_function=smoothie)
        review_bleu = bleu([label.strip().upper()], pred_s.strip().upper(), smoothing_function=smoothie)
        #print(pred_f, pred_s, first_bleu >= review_bleu)
        if first_bleu > review_bleu:
            print(round(first_bleu,3), round(review_bleu,3))
            print('정답 : ', label)
            print('한글 : ', hangle) 
            print('1차 답안 : ',pred_f)
            print('2차 답안 : ',pred_s)
            print()

0.073 0.05
정답 :  this isn’t  some sorta  scam, is it?
한글 :  사기꾼 아니야?
1차 답안 :  are you sure she’s not a fraud?
2차 답안 :  Isn't she a fraud?

0.417 0.17
정답 :  I didn’t  think people  would express their disappointment  so openly.
한글 :  이 정도로 노골적인 반응이 나올 줄이야….
1차 답안 :  I didn’t think people would be so blatantly rude...
2차 답안 :  I didn’t expect such an overt reaction...

0.753 0.747
정답 :  well, I’ll have to think about that later…
한글 :  뭐, 그건 천천히 고민해 보고….
1차 답안 :  well, I’ll think about that later...
2차 답안 :  Well, I’ll think about that later.

0.698 0.399
정답 :  …and focus on the task at hand for now.
한글 :  난 주어진 일만 처리하면 되겠지.
1차 답안 :  for now, I’ll just focus on the task at hand.
2차 답안 :  I’ll just handle the task given to me.

0.394 0.29
정답 :  the purpose behind  their punishment is equally important...
한글 :  사실 사건을 일으킨 용병을 벌하는 것만큼,
1차 답안 :  actually, the significance of the punishment...
2차 답안 :  Actually, punishing the mercenaries who caused the incident is just as important as the sign

In [49]:
#평가 데이터 제작 코드
import csv
for data_idx in range(58,68):#len(answers)):
    pred_df = pd.read_csv(f"data/simple_testset/result_{data_idx}.csv")
    hangles = pred_df['hangle']
    preds_first = pred_df['first_translate']  
    preds_second = pred_df['second_translate']
    labels = answers[data_idx]
    bleu_answers = []
    smoothie = SmoothingFunction().method4
    for label, pred_f, pred_s, hangle in zip(labels, preds_first, preds_second, hangles):
        first_bleu = bleu([label.strip().upper()], pred_f.strip().upper(), smoothing_function=smoothie)
        review_bleu = bleu([label.strip().upper()], pred_s.strip().upper(), smoothing_function=smoothie)
        if first_bleu > review_bleu:
            bleu_answers.append(1)
        elif first_bleu < review_bleu:
            bleu_answers.append(2)
        else:
            bleu_answers.append(3)
    with open(f"data/testset/compare_data_{data_idx}.csv", mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['원문', '첫 번 째 후보군', '두 번 째 후보군', 'bleu 기준 라벨'])
        for row in zip(hangles, preds_first, preds_second, bleu_answers):
            writer.writerow(row)
        # if first_bleu > review_bleu:
        #     print(round(first_bleu,3), round(review_bleu,3))
        #     print('정답 : ', label)
        #     print('한글 : ', hangle) 
        #     print('1차 답안 : ',pred_f)
        #     print('2차 답안 : ',pred_s)
        #     print()